![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


1. **First I find out what the top 3 industries are.**

In [12]:
SELECT 
    i.industry,
    COUNT(i.company_id) AS count_new_unicorns
FROM industries i
INNER JOIN dates as d 
USING(company_id)
WHERE EXTRACT(YEAR FROM d.date_joined) BETWEEN 2019 AND 2021
GROUP BY i.industry
ORDER BY count_new_unicorns DESC
LIMIT 3;

,industry,count_new_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


2. **Now that I have an idea for what to look for in my results I can start writing the querty to  identify the three best-performing industries based on the number of new unicorns created over the last three years (2019, 2020, and 2021) combined.**

In [36]:
WITH top_industries AS ( 
SELECT 
 i.industry,
 COUNT(i.company_id) AS count_new_unicorns
FROM industries i
INNER JOIN dates as d 
USING(company_id)
WHERE EXTRACT(YEAR FROM d.date_joined) BETWEEN 2019 AND 2021
GROUP BY i.industry
ORDER BY count_new_unicorns DESC
LIMIT 3
)
, yearly_rankings AS
( SELECT i.industry, EXTRACT(YEAR FROM d.date_joined) AS year, COUNT(company_id) AS num_unicorns, 
 AVG(f.valuation)  AS valuation_avg
FROM industries AS i
INNER JOIN dates AS d
USING (company_id)
INNER JOIN funding AS f
USING (company_id)
WHERE EXTRACT(YEAR FROM d.date_joined) BETWEEN 2019 AND 2021
GROUP BY i.industry, EXTRACT(YEAR FROM d.date_joined)
)

SELECT industry, year, num_unicorns, ROUND(AVG(valuation_avg / 1000000000),2) AS average_valuation_billions
FROM yearly_rankings 
INNER JOIN top_industries 
USING (industry)
GROUP BY yearly_rankings.industry, yearly_rankings.year, yearly_rankings.num_unicorns
ORDER BY industry ,year desc


,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
